In [1]:
# pandas의 활용이 가능한 경우에는 가급적 pandas를 연계하여 사용
import openpyxl
from openpyxl.utils import get_column_letter, column_index_from_string
from openpyxl.styles import Font
from openpyxl.styles import Alignment
from pathlib import Path
import random
import numpy as np
import win32com.client

In [2]:
p1 = Path.cwd() / 'attachments'
p2 = Path.cwd() / 'result_attachments'

In [3]:
# load_workbook으로 불러오기
# gspread의 경우 ss = gc.open('') 형태로 활용
wb = openpyxl.load_workbook(p1 / 'example.xlsx')
# sheetnames 메서드를 통해 시트들 확인 → 단순 문자열에 대한 리스트로 반환
# gspread의 경우 ss.worksheets() 형태로 활용 → worksheet 객체에 대한 리스트로 반환
wb.sheetnames

['Sheet1', 'Sheet2', 'Sheet3']

In [4]:
# active 메서드를 통해 활성 시트를 객체로 지정 가능
# 혹은 sheet = wb['Sheet1'] 형태로도 지정 가능
# gspread의 경우 ss.get_worksheet(0) 혹은 ss.worksheet('') 형태로 활용
sheet = wb['Sheet1']
# 현재 sheet의 제목 확인 가능
sheet.title

'Sheet1'

In [6]:
# 직접 접근하거나, cell(i, j).value를 이용하여 해당 셀 값 확인 가능
# gspread의 경우 acell('A1').value 혹은 cell(i, j).value 형태로 활용
# 셀 객체는 row, column, coordinate의 속성을 가짐 → sheet['A1'].coordinate('A1' 반환)
sheet.cell(1, 1).value, sheet['A1'].value, sheet['A1'].row, sheet['A1'].column, sheet['A1'].coordinate

(datetime.datetime(2015, 4, 5, 13, 34, 2),
 datetime.datetime(2015, 4, 5, 13, 34, 2),
 1,
 1,
 'A1')

In [8]:
# 열과 행의 최대 값이 어디인지 확인 가능 → 순환문에서 range의 끝으로 활용 등 다양한 용례
sheet.max_column, sheet.max_row

(9, 7)

In [12]:
# 열의 글자 이름과 숫자 이름의 변환을 위해 column_index_from_string() 및 get_column_letter() 사용
get_column_letter(1), column_index_from_string('B'), get_column_letter(sheet.max_column)

('A', 2, 'I')

In [14]:
# sheet['A1':'C3'] 형태로 시트에서 다수의 셀 객체 얻기 → 값이 아니라 각 셀 객체를 반환
# 순환문 활용해서 값들의 리스트 형식으로 가능(numpy의 array 형태와 유사하게 출력)
# gspread의 경우 sheet.get('A1:C3') 형태로 작성하면 그대로 값의 리스트 형식으로 출력
sheet['A1':'C3']

((<Cell 'Sheet1'.A1>, <Cell 'Sheet1'.B1>, <Cell 'Sheet1'.C1>),
 (<Cell 'Sheet1'.A2>, <Cell 'Sheet1'.B2>, <Cell 'Sheet1'.C2>),
 (<Cell 'Sheet1'.A3>, <Cell 'Sheet1'.B3>, <Cell 'Sheet1'.C3>))

In [17]:
for rowOfCellObj in sheet['A1':'C3']:
    for cellObj in rowOfCellObj:
        print(cellObj.coordinate, ':', cellObj.value)
    print('--------------')

A1 : 2015-04-05 13:34:02
B1 : Apples
C1 : 73
--------------
A2 : 2015-04-05 03:41:23
B2 : Cherries
C2 : 85
--------------
A3 : 2015-04-06 12:46:51
B3 : Pears
C3 : 14
--------------


In [21]:
cell_values = []
for rowOfCellObjects in sheet['A1':'C4']:
    for cellObj in rowOfCellObjects:
        cell_values.append(cellObj.value)
np.array(cell_values).reshape(-1, 3)

array([[datetime.datetime(2015, 4, 5, 13, 34, 2), 'Apples', 73],
       [datetime.datetime(2015, 4, 5, 3, 41, 23), 'Cherries', 85],
       [datetime.datetime(2015, 4, 6, 12, 46, 51), 'Pears', 14],
       [datetime.datetime(2015, 4, 8, 8, 59, 43), 'Oranges', 52]],
      dtype=object)

In [27]:
# rows 및 columns 속성을 사용하여 특정 행이나 열에 존재하는 셀 객체 접근
# list comprehension(여러 열의 경우 중첩 순환문) 활용하여 값의 리스트 형식 출력
# gspread의 경우 sheet.col_values(2), sheet.row_values(2)를 통해 작성하면 그대로 값의 리스트 형식으로 출력(B열과 2행 예시)
b_list = []
for cellObj in list(sheet.columns)[1]:
    b_list.append(cellObj.value)
b_list

['Apples', 'Cherries', 'Pears', 'Oranges', 'Apples', 'Bananas', 'Strawberries']

In [26]:
list(sheet.columns)[0], list(sheet.columns)[1]

((<Cell 'Sheet1'.A1>,
  <Cell 'Sheet1'.A2>,
  <Cell 'Sheet1'.A3>,
  <Cell 'Sheet1'.A4>,
  <Cell 'Sheet1'.A5>,
  <Cell 'Sheet1'.A6>,
  <Cell 'Sheet1'.A7>),
 (<Cell 'Sheet1'.B1>,
  <Cell 'Sheet1'.B2>,
  <Cell 'Sheet1'.B3>,
  <Cell 'Sheet1'.B4>,
  <Cell 'Sheet1'.B5>,
  <Cell 'Sheet1'.B6>,
  <Cell 'Sheet1'.B7>))

In [28]:
wb.close()

In [29]:
# 엑셀 문서 쓰기 : openpyxl.Workbook() 활용 → 괄호 안에는 아무것도 안들어가야(저장은 나중에 따로 경로 지정)
wb = openpyxl.Workbook()
# 하나의 시트에서 시작('Sheet')
wb.sheetnames

['Sheet']

In [30]:
sheet = wb.active
# sheet.title에 새로 지정을 하여 시트 이름 변경 가능
# gspread의 경우 따로 없는듯? 추후 조사 필요. ss.worksheet('name') 형태인가?
sheet.title = 'Food Sheet'
wb.sheetnames

['Food Sheet']

In [31]:
# create_sheet 활용하여 새로운 시트 생성 → index, title 인자 활용
# gspread의 경우 ss.add_worksheet(title='') 활용
wb.create_sheet(index=0, title='Sheet0')
wb.sheetnames, wb.active

(['Sheet0', 'Food Sheet'], <Worksheet "Sheet0">)

In [33]:
# del wb['Sheet0']의 형태로 시트 삭제 가능
# gspread의 경우 ss.del_worksheet(sheet) 활용 → sheet에는 sheet객체가 들어가야
try:
    del wb['Food Sheet']
except:
    print('없는 시트입니다.')
wb.sheetnames, wb.active

없는 시트입니다.


(['Sheet0'], <Worksheet "Sheet0">)

In [34]:
sheet['A1'] = 'Hello'
sheet.cell(2, 1).value = 'World'

In [35]:
# openpyxl.styles의 Font() 함수를 활용하여 폰트 지정 → 셀 객체에 font 속성으로 활용
# Font() → name, size, italic, bold 인자 전달 가능
italicBoldFont = Font(italic=True, bold=True)
sheet = wb.active
sheet.cell(1, 1).value = 'Hello'
sheet.cell(1, 1).font = italicBoldFont
sheet.cell(1, 1).value

'Hello'

In [38]:
italicBoldFont = Font(italic=False, bold=True)
sheet = wb.active
sheet.cell(2, 1).value = 'World'
sheet.cell(2, 1).font = italicBoldFont
sheet.cell(2, 1).value

'World'

In [42]:
# random.sample은 중복 없음 / random.choices는 중복 있음
# random.choices는 'k'라는 키워드 전달인자 필수 사용
randomNum = random.choices(range(10, 100), k=8)
for i, num in enumerate(randomNum):
    sheet.cell(i + 1, 2).value = num
# 엑셀의 수식 계산 함수(식)을 활용하여 openpyxl에서 셀에 수식 추가
sheet.cell(9, 2).value = '=sum(B1:B8)'
sheet['B9'].value

'=sum(B1:B8)'

In [43]:
wb.save(p2 / 'sample_excel.xls')

In [45]:
sheet['C1'].value = 'Tall row'
sheet['A2'].value = 'Wide column'
# row_dimensions[]와 column_dimensions[] 활용 -> height와 width 통해 행 높이, 열 너비(문자열) 조정 가능
sheet.row_dimensions[1].height = 70
sheet.column_dimensions['B'].width = 20

In [46]:
wb.save(p2 / 'sample_excel.xls')

In [47]:
# merge_cells 활용하여 셀 병합 + unmerge_cells를 활용하여 셀 병합 해제
sheet.merge_cells('E3:E5')

In [48]:
sheet.unmerge_cells('E3:E5')

In [49]:
# freeze_panes 활용하여 틀 고정 → 지정한 셀의 바로 앞에서 틀 고정(B열이면 A열 고정, 2행이면 1행 고정)
# freeze_panes = 'A1' 혹은 freeze_panes = None이면 틀 고정 해제
sheet.freeze_panes = 'B1'
sheet.freeze_panes = None
sheet.freeze_panes = 'B1'

In [51]:
wb.save(p2 / 'sample_excel.xls')
wb.close()

In [52]:
# 엑셀 차트 만들기
# 1. Reference 객체 생성(openpyxl.chart.Reference(sheet, min_col, min_row, max_col, max_row))
# 2. Reference 객체를 전달하여 Series 객체 생성(openpyxl.chart.Series(refObj, title=''))
# 3. Chart 객체 생성(openpyxl.chart.BarChart())
# 4. Chart 객체에 Series 객체 추가(chartObj.append(seriesObj))
# 5. sheet 객체에 Chart 객체 추가 → 차트의 좌측 상단 지점이 어느 셀에 위치할 지 지정(sheet.add_chart(chartObj, C5))
wb = openpyxl.Workbook()
sheet = wb.active
for i in range(1, 11):
    sheet.cell(i, 1).value = i
refObj = openpyxl.chart.Reference(sheet, min_row=1, min_col=1, max_row=10, max_col=1)
seriesObj = openpyxl.chart.Series(refObj, title='First Series')
chartObj = openpyxl.chart.BarChart()
chartObj.title = 'My Chart'
chartObj.append(seriesObj)
sheet.add_chart(chartObj, 'C5')
wb.save(p2 / 'sample_excel_chart.xls')
wb.close()

In [53]:
# win32com.client 모듈을 활용한 엑셀 컨트롤 → win32com.client.Dispatch('Excel.Application')
# win32com.client 모듈은 PascalCase 활용
excel = win32com.client.Dispatch('Excel.Application')
# excel.Visible = True 이용하여 엑셀을 보면서 컨트롤 가능
excel.Visible = True

In [65]:
# excel.Workbooks.Add() 혹은 excel.Workbooks.Open(경로)를 통해 엑셀 오픈
wb = excel.Workbooks.Add()
wb = excel.Workbooks.Open(p1 / 'example.xlsx')
# Worksheets(이름) 메서드 이용하여 시트 호출
ws = wb.Worksheets('Sheet1')

In [66]:
# ws.Rows(n).EntireRow.Insert()/Delete() 이용하여 행열 추가/삭제
ws.Rows(2).EntireRow.Insert()

True

In [67]:
ws.Rows(2).EntireRow.Delete()

True

In [68]:
ws.Columns(3).EntireColumn.Insert()

True

In [69]:
ws.Columns(3).EntireColumn.Delete()

True

In [70]:
# ws.Range(범위).Copy() 이후 ws.Range(범위).Select()로 옮길 범위 지정 후 ws.Paste()로 복사-붙여넣기
ws.Range('A1:C7').Copy()
ws.Range('G1:I7').Select()
# ws.Range(범위).PasteSpecial(n) 형태로 선택하여 붙여넣기
# -4122 : 서식 붙여넣기, -4123 : 수식 붙여넣기, -4163 : 값 붙여넣기, 11 : 수식과 표시 형식 붙여넣기, 12 : 값과 표시 형식 붙여넣기
ws.Range('G1:I7').PasteSpecial(-4163)

True

In [71]:
# ws.Range(범위).ClearContents()로 내용 삭제 가능
ws.Range('G1:I7').ClearContents()

True

In [72]:
ws.Range('A1:C7').Copy()
ws.Range('G1').Select()
ws.Range('G1').PasteSpecial(-4163)

True

In [73]:
ws.Range('G1:I7').ClearContents()

True

In [74]:
ws.Range('E1:F1').Value = 1
ws.Range('E2:F2').Value = 2

In [75]:
# ws.Range('F1:F2').AutoFill(ws.Range('F1:F7')) 형태로 값 및 수식 자동 채우기
ws.Range('E1').AutoFill(ws.Range('E1:E7'))
ws.Range('F1:F2').AutoFill(ws.Range('F1:F7'))

True

In [76]:
# Save() 및 Close() 메서드를 활용하여 저장 및 종료
wb.Save()
wb.Close()
# excel.Quit() 이용하여 엑셀파일 종료
excel.Quit()